<div align="center">
  <img src="http://vlpavlov.org/Pythagoras-Logo3.svg"><br>
</div>

# Pythagoras
## Introduction

This tutorial explains the core Pythagoras constructs, which allow everyone to easily parallelize their code and execute it in the cloud with just a few lines of extra code.

### Initial setup

First, let's install Pythagoras and import FileDirDict class:

In [128]:
!pip install pythagoras --quiet

In [129]:
from pythagoras import *

### Hello, World! 

Two key classes we need to create a basic Pythagoras program:
    
* **ServerlessCloud**: objects of this class are responsible for actuall connection to the cloud (AWS, GCP, Azhure, etc.)
* **CloudModule**: objects of thios class are capable to store and execute in the cloud parallelized versions of your functions

In [132]:
my_cloud = ServerlessCloud("some parameters")
my_cloud_module = CloudModule(cloud=my_cloud)

In [146]:
@my_cloud_module.add
def very_slow_function(*,very_important_parameter:int):
    """This function runs for about an hour"""
    return very_important_parameter**2

@my_cloud_module.add
def another_slow_function(*,best_ever_parameter:int):
    """This function runs for about an hour"""
    return very_important_parameter**3

Once we added all our slow functions to the model, we need to "finalize" it. This action will push all regestered to the cloud and use all the benefits of seamless access to serverless compute

In [147]:
my_cloud_module.finalize()

There are three main benefits of turning your regular function to a cloud-hosted function:
* Cloud-based memoization
* Cloud-based execution
* Cloud-based parallelization

Let's take a closer look:

### Cloud-based memoization

In [148]:
very_slow_function(very_important_parameter=2)

4

In [149]:
very_slow_function(very_important_parameter=2)

4

In [150]:
very_slow_function(very_important_parameter=10)

100

### Cloud-based execution

In [134]:
very_slow_function.remote( very_important_parameter=12345 )

1

In [ ]:
very_slow_function.remote( very_important_parameter=12345 )

In [ ]:
very_slow_function.remote( very_important_parameter=2)

### Cloud-based parallelization

In [151]:
results = []
for i in range(5):
    results.append(very_slow_function( very_important_parameter=i ))
results

[0, 1, 4, 9, 16]

In [152]:
[very_slow_function(very_important_parameter=i) for i in range(5)]

[0, 1, 4, 9, 16]

In [153]:
very_slow_function.parallel(remote(very_important_parameter=i) for i in range(5))

[9, 0, 1, 16, 4]

### Summary of key capabilities

By adding a simple decorator in front of your Python function, you can turn it into a serverless code that can run both locally and remotely. Another line of code replaces sequential loops with a parallel execution engine that simultaneously launches hundreds of serverless functions in the cloud. This is a perfect solution for complex computational tasks, such as multi-fold cross-validation, grid search for hyperparameter optimization, or feature selection algorithms.

For pure functions (fully deterministic, no side-effect functions whose output values depend solidly on input values), Pythagoras provides cloud storage to cache function outputs. Such memoized functions run only once, all subsequent calls on any computer will skip function execution and return previously computed values. It makes complex distributed algorithms cheap to rerun, and easy to resume in case they were interrupted.

Cloud storage is partially replicated on local computers, which allows Python scripts and notebooks to access stored values very fast. Each piece of data is associated with its hash that serves as a key to access the data. When some data (e.g., a large Pandas DataFrame) must be passed as an input to a serverless function, under the hood Pythagoras pushes the data to the cloud storage, and only passes its hash to the function. This approach optimizes traffic, associated with launching new instances of serverless functions in the cloud, and significantly speeds up the process.

The typical scenario of working with Pythagoras is to parallelize Python code using backend compute infrastructure, provided by a major cloud vendor. We are currently working on creating reference implementation for AWS, with plans to integrate with GCP and Azhure later. As an alternative, Pythagoras offers a simple P2P model, in which serverless code can be parallelized over a distributed swarm of workstations, on-premise servers, and even laptops. This model is a good solution for resource constrained teams and educational projects.